# Using an Echo-State Network to predict ENSO

## Preparations

In [1]:
cd("$(homedir())/Documents/enso_project.jl")
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\lihel\Documents\enso_project.jl`


In [2]:
using ReservoirComputing, CSV, DataFrames, DynamicalSystems, Plots, enso_project

┌ Warning: Circular dependency detected. Precompilation will be skipped for:
│   Base.PkgId(Base.UUID("e3ecd195-ca82-5397-9546-f380c1e34951"), "NonlinearSolveBaseSparseMatrixColoringsExt")
│   Base.PkgId(Base.UUID("385e4588-a1a0-5c1d-98fa-d45bf6f8ecf9"), "LinearSolveKernelAbstractionsExt")
│   Base.PkgId(Base.UUID("3bcf3b12-2128-5d18-8b3b-bcdd6f83637b"), "WeightInitializersGPUArraysExt")
│   Base.PkgId(Base.UUID("b00db79b-61e3-50fb-b26f-2d35b2d9e4ed"), "DiffEqBaseChainRulesCoreExt")
│   Base.PkgId(Base.UUID("8913a72c-1f9b-4ce2-8d82-65094dcecaec"), "NonlinearSolve")
│   Base.PkgId(Base.UUID("7edab7de-1038-5e4f-97a7-6bfc75d44324"), "NonlinearSolveQuasiNewtonForwardDiffExt")
│   Base.PkgId(Base.UUID("1dea7af3-3e70-54e6-95c3-0bf5283fa5ed"), "OrdinaryDiffEq")
│   Base.PkgId(Base.UUID("0d7ed370-da01-4f52-bd93-41d350b8b718"), "StaticArrayInterface")
│   Base.PkgId(Base.UUID("693f0f32-89f9-59b4-b981-3d79b82ef24b"), "SparseDiffToolsEnzymeExt")
│   Base.PkgId(Base.UUID("63d416d0-6995-5965-81e0-5

ErrorException: Failed to precompile enso_project [92f8d58f-31a9-439a-b05e-792ebdf1e7ff] to "C:\\Users\\lihel\\.julia\\compiled\\v1.11\\enso_project\\jl_9DE6.tmp".

In [ ]:
# read and format input data for all splits
splits = [20,40,50,60,70,80]
data = Dict()

for s in splits

    # set dict keys
    key_train = "train_data_$s"
    key_test = "test_data_$s"
    key_val = "val_data_$s"

    # read input data
    data[key_train] = CSV.read("data/sst_34_data_split_$s/train_sst_34_anomaly_embedded_$s.txt", DataFrame; delim=',', ignorerepeated=true)
    data[key_test] = CSV.read("data/sst_34_data_split_$s/test_sst_34_anomaly_embedded_$s.txt", DataFrame; delim=',', ignorerepeated=true)
    data[key_val] = CSV.read("data/sst_34_data_split_$s/val_sst_34_anomaly_embedded_$s.txt", DataFrame; delim=',', ignorerepeated=true)
   
    # bring into correct format
    data[key_train] = Matrix(transpose(Matrix(data[key_train])))
    data[key_test] = Matrix(transpose(Matrix(data[key_test])))
    data[key_val] = Matrix(transpose(Matrix(data[key_val])))
end

data

## Network Training on Different Data Splits
We train the network in the same manner for six different data splits. For each data split, we try our a set of different hyperparameters.

### Hyperparameter Tuning
For each data split, we choose suitable hyperparameters by performing a grid search.


We create the hyperparameter grid as follows:
- We observe that too big reservoir sizes cause singular matrices in the linear regression (due to too little training data), thus we adapt the reservoir size to be adequately small compared to the amount of training data. This means, that the reserovir sizes are changed in each data split.
- to ensure the Echo State Property the spectral radius should be smaller than 1 (unless long memory is required). We try out different combinations.
- a sparsity of 0.1 is usually recommended, we test different values around 0.1
- a input scale of 0.1 is recommended by the literature
- we try ridge params as suggested in the lecture

In [ ]:
# set up universal parameter options of parameter grid
spectral_radii = [0.8, 0.9, 1.0]
sparsities = [0.05, 0.08, 0.1, 0.12, 0.14]
input_scales = [0.1]
ridge_values = [0.0, 1e-6, 1e-5]

### Training Data Split 20%

train = 20%,
val =  70%,
test = 10%

We see that smaller reservoir sizes around 10 are favoured and adapt the parameter grid accordingly.

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 40]

param_grid_20 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)

length(param_grid_20)

In [ ]:
# network training
esn_20, W_out_20 = enso_project.cross_validate_esn(data["train_data_20"], data["val_data_20"], param_grid_20)

the validation loss is quite high after training.

In [ ]:
enso_project.plot_esn_prediction(esn_20, W_out_20, data["test_data_20"])

a visual analysis also yields that the perfomance is not good. We need more training data for a good prediction.

### Training Data Split 40%

train = 40%,
val =  50%,
test = 10%

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [9, 10, 12, 14, 16, 18, 20, 30, 40, 50]
param_grid_40 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
length(param_grid_40)

In [ ]:
# network training
esn_40, W_out_40 = enso_project.cross_validate_esn(data["train_data_40"], data["val_data_40"], param_grid_40)

the validation loss is significantly smaller than in the previous 20% split, but still high

In [ ]:
enso_project.plot_esn_prediction(esn_40, W_out_40, data["test_data_40"])

We see that the prediction is still not very accurate, although the dynamics are captured a bit better.

### Training Data Split 50%

train = 50%,
val =  40%,
test = 10%

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [9, 10, 12, 14, 16, 18, 20, 30, 40, 50]
param_grid_50 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
length(param_grid_50)

In [ ]:
# network training
esn_50, W_out_50 = enso_project.cross_validate_esn(data["train_data_50"], data["val_data_50"], param_grid_50)

the validation loss decreased a bit.

In [ ]:
enso_project.plot_esn_prediction(esn_50, W_out_50, data["test_data_50"])

The prediciton captures the dynamics of the system quite well for approximately 30 months.

### Training Data Split 60%

train = 60%,
val =  30%,
test = 10%

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [10, 14, 15, 18, 20, 22, 24, 26, 28, 30, 35, 40, 50]
param_grid_60 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
length(param_grid_60)

In [ ]:
# network training
esn_60, W_out_60 = enso_project.cross_validate_esn(data["train_data_60"], data["val_data_60"], param_grid_60)

The more data is used for training, the more the validation loss decreases

In [ ]:
enso_project.plot_esn_prediction(esn_60, W_out_60, data["test_data_60"])

The predicitons for the first 30 months get more accurate.

### Training Data Split 70%

train = 70%,
val =  20%,
test = 10%

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [ 40, 50, 60, 62, 63, 64, 65, 66, 68, 70, 72, 74, 80, 90]
param_grid_70 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
length(param_grid_70)

In [ ]:
# network training
esn_70, W_out_70 = enso_project.cross_validate_esn(data["train_data_70"], data["val_data_70"], param_grid_70)

In [ ]:
enso_project.plot_esn_prediction(esn_70, W_out_70, data["test_data_70"])

### Training Data Split 80%

train = 80%,
val =  10%,
test = 10%

In [ ]:
# set reservoir sizes according to amount of training data
reservoir_sizes = [60, 70, 80, 90, 100, 110, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 150]
param_grid_80 = enso_project.create_param_grid(reservoir_sizes, spectral_radii, sparsities, input_scales, ridge_values)
length(param_grid_80)

In [ ]:
# network training
esn_80, W_out_80 = enso_project.cross_validate_esn(data["train_data_80"], data["val_data_80"], param_grid_80)

very low validation loss for set of hyperparameters $(130,0.9,0.14,0.1,1.0e-5)$

In [ ]:
enso_project.plot_esn_prediction(esn_80, W_out_80, data["test_data_80"])

very accurate prediction for up to 20 months.

In [ ]:
# plot against validation data
enso_project.plot_esn_prediction(esn_80, W_out_80, data["val_data_80"])

In [ ]:
# plot against training data
enso_project.plot_esn_prediction(esn_80, W_out_80, data["train_data_80"])

## Comparison of Data Splits

put here plots of validation loss evolvement and one figure with all subplots

TODO:
- find a way to store best validation loss after cross validation
- store all 6 losses in an array and line-plot

## Evaluation of Prediction Accuracy